<a href="https://colab.research.google.com/github/liangli217/PyTorch_ML/blob/main/CNN_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem: Implement a CNN for CIFAR-10 in PyTorch

### Problem Statement
You are tasked with implementing a **Convolutional Neural Network (CNN)** for image classification on the **CIFAR-10** dataset using PyTorch. The model should contain convolutional layers for feature extraction, pooling layers for downsampling, and fully connected layers for classification. Your goal is to complete the CNN model by defining the necessary layers and implementing the forward pass.

### Requirements
1. **Define the CNN Model**:
   - Add **convolutional layers** for feature extraction.
   - Add **pooling layers** to reduce the spatial dimensions.
   - Add **fully connected layers** to output class predictions.
   - The model should be capable of processing input images of size `(32x32x3)` as in the CIFAR-10 dataset.

### Constraints
- The CNN should be designed with multiple convolutional and pooling layers followed by fully connected layers.
- Ensure the model is compatible with the CIFAR-10 dataset, which contains 10 classes.


<details>
  <summary>💡 Hint</summary>
  Add the convolutional (conv1, conv2), pooling (pool), and fully connected layers (fc1, fc2) in CNNModel.__init__.
  <br>
  Implement the forward pass to process inputs through these layers.
</details>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [2]:
# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

100%|██████████| 170M/170M [00:06<00:00, 26.5MB/s]


In [3]:
for images, labels in train_loader:
    print(images.shape, labels.shape)
    break

torch.Size([64, 3, 32, 32]) torch.Size([64])


In [4]:

# Define the CNN Model
# TODO: Add convolutional, pooling, and fully connected layers
class CNNModel(nn.Module):
    def __init__(self):

      super().__init__()
      self.conv1 = nn.Conv2d(3, 32, kernel_size = 3, padding = 1)
      self.conv2 = nn.Conv2d(32, 64, kernel_size = 3, padding = 1)
      self.pool = nn.MaxPool2d(2, 2)
      self.relu = nn.ReLU()
      self.fc1 = nn.Linear(64 * 16* 16, 512)
      self.fc2 = nn.Linear(512, 10)

    def forward(self, x):

      x = self.conv1(x)
      x = self.relu(x)
      x = self.conv2(x)
      x = self.relu(x)
      x = self.pool(x)
      x = x.view(x.size(0), -1)
      x = self.fc1(x)
      x=  self.relu(x)
      x = self.fc2(x)

      return x



In [5]:
# Initialize the model, loss function, and optimizer
model = CNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10
for epoch in range(epochs):
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [1/10], Loss: 0.8662
Epoch [2/10], Loss: 0.4851
Epoch [3/10], Loss: 0.7667
Epoch [4/10], Loss: 0.1808
Epoch [5/10], Loss: 0.0275
Epoch [6/10], Loss: 0.0283
Epoch [7/10], Loss: 0.0788
Epoch [8/10], Loss: 0.0388
Epoch [9/10], Loss: 0.0021
Epoch [10/10], Loss: 0.0031


In [32]:
# TODO: Save the model to a file named "model.pth"
saved_model = torch.save(model.state_dict(), "CNN_model.pth")

# TODO: Load the model back from "model.pth"
loaded_model = CNNModel()
loaded_model.load_state_dict(torch.load("CNN_model.pth"))

<All keys matched successfully>

In [59]:
correct = 0
total = 0
for images, labels in test_loader:
        outputs = loaded_model(images)
        total += labels.size(0)

        predicted_labels = torch.max(outputs, 1).indices
        correct += torch.sum(predicted_labels==labels).item()
correct/total

0.6926

In [53]:
# Evaluate on the test set
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 69.26%


In [ ]:
x_flat = x.view(x.size(0), -1)
x_flat.shape

torch.Size([16, 300])